In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import plotly.express as px

In [ ]:
%run ../modules/utils.ipynb
%run ../modules/cds.ipynb
%run ../modules/preprocessing.ipynb

In [ ]:
df7 = pd.read_excel('/Users/bryan/Documents/BioPharmaFinder/BaoLab/Excels/S6to10combined_20ul_40min_std.xlsx')

In [ ]:
plot_zone(df7)

In [ ]:
df7 = df7.dropna()

In [ ]:
df7.info()

In [ ]:
df_big = df7[df7.Mass>22000].sort_values('Vol', ascending=False)[:10][['Mass', 'Vol', 'RT']]
df_big # 0, 

In [ ]:
# full_mass = df_big.iloc[0].Mass
# full_mass = 23277.1 #23281.1 #24206.2 #24330.2261
# full_mass = 23595.15 #24234.26
full_mass = 24287.20662#23057.09
print(full_mass)
df_ms, _ = computational_data_seperation(df7, df7, full_mass=full_mass, ignore_endpoints=True)
df_ms.shape
plot_zone(df_ms)

In [ ]:
df_ms_bcr, mass_pairs = base_calling_random(df_ms)
df_ms_bcr.shape, len(mass_pairs)
plot_basecalling(df_ms_bcr, mass_pairs, annotate=False)

In [ ]:
mass_3p = [pair[0] for pair in mass_pairs]
mass_5p = [pair[1] for pair in mass_pairs]
mass_3p5p = set(mass_3p) & set(mass_3p)
mass_3ponly = set(mass_3p) - set(mass_5p)
mass_5ponly = set(mass_5p) - set(mass_3p)
mass_else = mass_3ponly | mass_5ponly
len(mass_3p5p), len(mass_else)

df_mass_3p5p = df_ms_bcr[df_ms_bcr.Mass.isin(mass_3p5p)]
# plot_zone(df_mass_3p5p)
# plot_zone(df_ms_bcr[df_ms_bcr.Mass.isin(mass_else)])
plt.figure(figsize=(12, 9))
plt.xlabel('Mass')
plt.ylabel('RT')
plt.scatter(df_mass_3p5p.Mass, df_mass_3p5p.RT)
# plt.savefig('/Users/bryan/Documents/BioPharmaFinder/BaoLab/Excels/S6to10combined_20ul_40min_skeleton_1.png', transparent=True)

In [ ]:
# df_mass_3p5p.to_excel('/Users/bryan/Documents/BioPharmaFinder/BaoLab/Excels/S6to10combined_20ul_40min_skeleton_1.xlsx')

In [ ]:
fig = px.scatter(x=df7.Mass, y=df7.RT)
fig.show()

#    Rank to get the full mass value

In [ ]:
df_glu = df7[df7.Mass > 1500]

In [ ]:
df3p_mass_np = np.array(df_glu['Mass'])
df5p_mass_np = np.array(df_glu['Mass'])
mass_sum_np = df3p_mass_np[:, np.newaxis] + df5p_mass_np
mass_sum_0 = np.round(mass_sum_np, 0)
mass_sum_1 = np.round(mass_sum_np, 1)
mass_sum_2 = np.round(mass_sum_np, 2)
mass_sum_3 = np.round(mass_sum_np, 3)

In [ ]:
res1 = np.unique(mass_sum_1, return_counts=True)
res2 = np.unique(mass_sum_2, return_counts=True)
res3 = np.unique(mass_sum_3, return_counts=True)

In [ ]:
res1_s0 = pd.Series(res1[0])
res1_s1 = pd.Series(res1[1])

res2_s0 = pd.Series(res2[0])
res2_s1 = pd.Series(res2[1])

res3_s0 = pd.Series(res3[0])
res3_s1 = pd.Series(res3[1])

In [ ]:
res1_s1_max = np.max(res1_s1)
res1_s2_max = np.max(res2_s1)
res1_s3_max = np.max(res3_s1)
print("max s1 {} s2 {} s3 {}".format(res1_s1_max, res1_s2_max, res1_s3_max))

In [ ]:
res1_idxs = res1_s1[res1_s1 > 400].index
res1_s0[res1_idxs]
l = list(zip(res1_s0[res1_idxs], res1_s1[res1_idxs]))
[i for i in l if i[0] > 23000]

In [ ]:
res2_idxs = res2_s1[res2_s1 > 80].index
l = list(zip(res2_s0[res2_idxs], res2_s1[res2_idxs]))
[i for i in l if i[0] > 23000]

In [ ]:
res3_idxs = res3_s1[res3_s1 > 10].index
l = list(zip(res3_s0[res3_idxs], res3_s1[res3_idxs]))

# Top vols

In [ ]:
plot_zone(df_glu)

In [ ]:
df_glu = df_glu.sort_values('Vol', ascending=False)

In [ ]:
df_top = df_glu[(df_glu.Mass > 11000) & (df_glu.Mass < 20000)][:500]

In [ ]:
fig = px.scatter(x=df_top.Mass, y=df_top.RT)
fig.show()

In [ ]:
plot_zone(df_top)

# Top Vol

In [ ]:
import plotly.graph_objects as go

idxs = list()
# create our callback function
def update_point(trace, points, selector):
    print('points {}'.format(points.point_inds) )
    idxs.extend(points.point_inds)

df_draw = df7[(df7.Mass > 5000) & (df7.Mass < 22000)]
# df_draw = df7_chosen[:100] #df7
x = df_draw.Mass
y = df_draw.RT
fig = go.Scatter(x=x, y=y, mode='markers')
# fig.update_layout(width=1024, height=768)
f = go.FigureWidget(fig)
scatter = f.data[0]
scatter.on_selection(update_point)
f

In [ ]:
df7_chosen = df_draw.iloc[idxs]
df7_chosen.info()

In [ ]:
df7_chosen = df7_chosen.sort_values('Vol', ascending=False)

In [ ]:
bcr_5p = base_calling_random(df7_chosen[:100])
plot_basecalling(*bcr_5p, annotate=True)

In [ ]:
plot_zone(df7_chosen)

In [ ]:
df7_chosen.to_excel('/Users/bryan/Documents/BioPharmaFinder/BaoLab/Excels/S6to10combined_20ul_40min_skeleton_2.xlsx')

In [ ]:
# df_glu_top = df_glu_kernel[:500]
fig = px.scatter(x=df7_chosen.Mass, y=df7_chosen.RT, width=1024, height=768)
fig.show()

# Grouped RT, Top Vol

In [ ]:
df7_top = df7[(df7.Mass > 2000) & (df7.Mass < 22000)].sort_values('Vol').groupby('RT').tail(1)

In [ ]:
import plotly.graph_objects as go

idxs = list()
# create our callback function
def update_point(trace, points, selector):
    print('points {}'.format(points.point_inds) )
    idxs.extend(points.point_inds)

# df_draw = df2[(df2.Mass > 12329) & (df2.Mass < 22000) & (df2.RT < 25) & (df2.RT > 8)]
df_draw = df7_chosen[(df7_chosen.Mass > 6000) & (df7_chosen.Mass < 15000) & (df7_chosen.RT < 25) & (df7_chosen.RT > 10)] #df7
# df_draw = df7_top[(df7_top.RT < 25) & (df7_top.RT > 8)]
x = df_draw.Mass
y = df_draw.RT
fig = go.Scatter(x=x, y=y, mode='markers')
f = go.FigureWidget(fig)
f.update_layout(autosize=False, width=1024, height=700, paper_bgcolor="LightSteelBlue",
    margin=dict(
        l=20,
        r=20,
        b=20,
        t=20,
        pad=4
    ))
scatter = f.data[0]
scatter.on_selection(update_point)
f

In [ ]:
df7_chosen = df_draw.iloc[idxs]
df7_chosen.info()
df7_chosen = df7_chosen.sort_values('Vol', ascending=False)

In [ ]:
# plot_zone(df2_chosen[:100])
bcr_5p = base_calling_random(df7_chosen[(df7_chosen.Mass>5800) & (df7_chosen.Mass < 13000)])
plt = plot_basecalling(*bcr_5p, annotate=True)
# plt.savefig('/Users/bryan/Downloads/Glu_7.png', bbox_inches='tight', transparent=True)